---
title: "Race Report"
author: "Janik Wahrheit"
format:
  html:
    page-layout: full
    header-includes:
      - |
        <style>
          /* Targeting the Quarto header class */
          .quarto-header {
            background-color: red;  /* Set background color to red */
            color: white;            /* Set text color to white */
            padding: 20px;
            text-align: center;
          }

          .quarto-header .quarto-title {
            color: white;  /* Ensure the title text is white */
          }

          .quarto-header .quarto-author {
            color: white;  /* Ensure the author text is white */
          }
        </style>
---



In [32]:
#| echo: false
from dataservice import DataService

In [33]:
#| echo: false
#| output: false
obj = DataService()
obj.set_source("https://my.raceresult.com/309137/")
obj.handle_cookie_banner() 
obj.click_show_all_buttons() 
obj.create_soup() 
obj.get_metadata() 
obj.expand_table() 
obj.get_table_as_df() 

[DEBUG] DataService initialized.
[DEBUG] Navigated to URL: https://my.raceresult.com/309137/
[DEBUG] Source set to: https://my.raceresult.com/309137/
[DEBUG] Current source: https://my.raceresult.com/309137/
Cookie-Banner geschlossen.


In [34]:
#| echo: false
from IPython.display import Markdown, display

metadata_md = f"""
# RaceReport - {obj.title}

| **Date Range**          | **Location**           |
|--------------------------|------------------------|
| {obj.date}              | {obj.location}        |
"""
display(Markdown(metadata_md))





# RaceReport - Winterlaufserie Rheinzabern

| **Date Range**          | **Location**           |
|--------------------------|------------------------|
| 15.12.2024-09.02.2025              | Rheinzabern, Deutschland        |


In [35]:
obj.dataFrame

,Pl.,Snr.,Name,Nat.,Jg.,Verein,AK-Pl.,MW-Pl.,Zeit
0,1.,1383,BECKER Jan Lukas,,1993,Sport Jochum,1. M30,1. M,29:25
1,2.,1481,GROßKOPF Julian,,2000,LAZ Ludwigsburg,1. M,2. M,30:21
2,3.,1109,ZEHFELD Lennart,,2005,LC Rehlingen,2. M,3. M,30:41
3,4.,1060,TESF AMARIAM ESTOPIA Selama,,1995,TV Alzey Laufteam,2. M30,4. M,30:47
4,5.,1253,BAUER Clemens,,2006,TV Waldstraße Wiesbaden,1. MJ U20,5. M,30:57
...,...,...,...,...,...,...,...,...,...
1332,1333.,250,HOTZ Nadine,,1981,,52. W40,439. W,1:18:06
1333,1334.,629,VÖLKER Bernd,,1959,Marathon-Team Ketsch,31. M65,895. M,1:18:53
1334,1335.,1298,FREY Rolf-Jürgen,,1955,Marathon-Team Ketsch,19. M70,896. M,1:23:12
1335,1336.,1411,WIEDENMANN Walter,,1947,LT Altburg,7. M75,897. M,1:26:10
